In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold 
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_examples_split=2, max_depth=100, n_features=None):
        self.min_examples_split = min_examples_split
        self.max_depth = max_depth
        # This is to only pick a number of features to train instead of all of the features
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_examples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Check the stopping criteria
        if depth >= self.max_depth or n_labels == 1 or n_examples < self.min_examples_split:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # total number of features that we have; number of features that we want to select from our object; false makes it select only unique 
        # features
        feat_idxs = np.random.choice(n_features, self.n_features, replace=False)
        
        # Find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # Create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)

        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)

        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self.information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold
    
    def information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0 

        # calculate the weighted avg. entropy of children
        n = len(y)

            # number of examples
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
            # nl_n / n, nr_n / n are weighted averages; it means how many examples are in one and in the other vector
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_dxs = np.argwhere(X_column > split_thresh).flatten()

        return left_idxs, right_dxs
    
    def _entropy(self, y):
        # count the number of occurences of each number starting from 0 up until the last int in list incldued
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])



    def _most_common_label(self, y):
        counter = Counter(y)
        # Get the most common label; get the most common tuple; first information that includes the value
        # value = counter.most_common(1)[0]
        value = counter.most_common(1)[0][0]
        # print(counter.most_common(1))
        # print(counter.most_common(1)[0])
        # print(counter.most_common(1)[0][0])
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split


In [21]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target
print(X.shape)
print(type(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = DecisionTree()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

(569, 30)
<class 'numpy.ndarray'>


In [5]:
def accuracy(y_test, y_pred):
    # y_test == y_pred return 0 or 1 and thus, we sum the 1s, which indicate the number of examples guessed correctly
    return np.sum(y_test == y_pred) / len(y_test)
    # return np.sum(y_pred == y_test) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9210526315789473


In [19]:
df = pd.read_csv("winequality-red.csv")

df.dropna(inplace=True)


In [ ]:
X = np.array(df.drop(columns="quality"))
y = np.array(df["quality"])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# clf = DecisionTree()
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_test)

array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
       [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
       [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]])

0       0.00
1       0.00
2       0.04
3       0.56
4       0.00
        ... 
1594    0.08
1595    0.10
1596    0.13
1597    0.12
1598    0.47
Name: citric acid, Length: 1599, dtype: float64